In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import pipeline
import re
from googletrans import Translator

In [6]:
hindi_stopwords = [
    "और", "का", "के", "की", "हैं", "है", "था", "थे", "से", "में", "को", "पर",
    "नहीं", "हुआ", "हुई", "हो", "कर", "दिया", "रहा", "करना", "करते", "सकते",
    "इस", "होता", "जाता", "होते", "भी", "बाद", "लिए", "आप", "जो", "कहा",
    "जब", "तो", "क्या", "जी", "ये", "जी", "वह", "जैसे", "कई", "कहते", "गया",
    "करें", "होती", "अपना", "उनके"
]

In [7]:
conversation = """
Recovery Agent (RA): नमस्ते श्री कुमार, मैं एक्स वाई जेड फाइनेंस से बोल रहा हूं। आपके लोन के बारे में बात करनी थी।
Borrower (B): हां, बोलिए। क्या बात है?
RA: सर, आपका पिछले महीने का EMI अभी तक नहीं आया है। क्या कोई समस्या है?
B: हां, थोड़ी दिक्कत है। मेरी नौकरी चली गई है और मैं नया काम ढूंढ रहा हूं।
RA: ओह, यह तो बुरा हुआ। लेकिन सर, आपको समझना होगा कि लोन का भुगतान समय पर करना बहुत जरूरी है।
B: मैं समझता हूं, लेकिन अभी मेरे पास पैसे नहीं हैं। क्या कुछ समय मिल सकता है?
RA: हम समझते हैं आपकी स्थिति। क्या आप अगले हफ्ते तक कुछ भुगतान कर सकते हैं?
B: मैं कोशिश करूंगा, लेकिन पूरा EMI नहीं दे पाऊंगा। क्या आधा भुगतान चलेगा?
RA: ठीक है, आधा भुगतान अगले हफ्ते तक कर दीजिए। बाकी का क्या प्लान है आपका?
B: मुझे उम्मीद है कि अगले महीने तक मुझे नया काम मिल जाएगा। तब मैं बाकी बकाया चुका दूंगा।
RA: ठीक है। तो हम ऐसा करते हैं - आप अगले हफ्ते तक आधा EMI जमा कर दीजिए, और अगले महीने के 15 तारीख तक बाकी का भुगतान कर दीजिए। क्या यह आपको स्वीकार है?
B: हां, यह ठीक रहेगा। मैं इस प्लान का पालन करने की पूरी कोशिश करूंगा।
RA: बहुत अच्छा। मैं आपको एक SMS भेज रहा हूं जिसमें भुगतान की डिटेल्स होंगी। कृपया इसका पालन करें और समय पर भुगतान करें।
B: ठीक है, धन्यवाद आपके समझने के लिए।
RA: आपका स्वागत है। अगर कोई और सवाल हो तो मुझे बताइएगा। अलविदा।
B: अलविदा।
"""


In [8]:
#Text Cleaning removing extra white spaces
conversation = re.sub(r'\s+', ' ', conversation)


In [9]:
#tokenization to break down text into single sentences
sentences = sent_tokenize(conversation)

In [10]:
#Speaker Identification
tagged_sentences = []
for sentence in sentences:
    if 'RA:' in sentence:
        tagged_sentences.append(('RA', sentence.replace('RA:', '').strip()))
    elif 'B:' in sentence:
        tagged_sentences.append(('B', sentence.replace('B:', '').strip()))

In [11]:
# Lemmatization and Stop Words Removal
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MohdS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MohdS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
processed_sentences = []
for speaker, sentence in tagged_sentences:
    words = word_tokenize(sentence)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in hindi_stopwords]
    processed_sentences.append((speaker, ' '.join(filtered_words)))

In [13]:
# Print processed sentences
for speaker, sentence in processed_sentences:
    print(f'{speaker}: {sentence}')

RA: सर , आपका पिछले महीने EMI अभी तक आया है। कोई समस्या ?
RA: B : हां , थोड़ी दिक्कत है। मेरी नौकरी चली गई मैं नया काम ढूंढ हूं। ओह , यह बुरा हुआ। लेकिन सर , आपको समझना होगा कि लोन भुगतान समय बहुत जरूरी है। B : मैं समझता हूं , लेकिन अभी मेरे पास पैसे हैं। कुछ समय मिल सकता ?
RA: हम समझते आपकी स्थिति। अगले हफ्ते तक कुछ भुगतान ?
B: मैं कोशिश करूंगा , लेकिन पूरा EMI दे पाऊंगा। आधा भुगतान चलेगा ?
RA: ठीक , आधा भुगतान अगले हफ्ते तक दीजिए। बाकी प्लान आपका ?
RA: B : मुझे उम्मीद कि अगले महीने तक मुझे नया काम मिल जाएगा। तब मैं बाकी बकाया चुका दूंगा। ठीक है। हम ऐसा - अगले हफ्ते तक आधा EMI जमा दीजिए , अगले महीने 15 तारीख तक बाकी भुगतान दीजिए। यह आपको स्वीकार ?
RA: B : हां , यह ठीक रहेगा। मैं प्लान पालन करने पूरी कोशिश करूंगा। बहुत अच्छा। मैं आपको एक SMS भेज हूं जिसमें भुगतान डिटेल्स होंगी। कृपया इसका पालन समय भुगतान करें। B : ठीक , धन्यवाद आपके समझने लिए। आपका स्वागत है। अगर कोई सवाल मुझे बताइएगा। अलविदा। B : अलविदा।


In [14]:
# Initialize the translator
translator = Translator()

In [15]:
# Translate the conversation to English
translation = translator.translate(conversation, src='hi', dest='en')

In [26]:
translated_text = translation.text

'Recovery Agent (RA): Namaste Mr. Kumar, I am speaking from X Y Z Finance.Had to talk about your loan.Borrower (b): Yes, speak.What is the matter?Ra: Sir, your EMI of your previous month has not come yet.are there any problems?B: Yes, there is some problem.My job is gone and I am looking for new work.Ra: Oh, this is bad.But Sir, you have to understand that it is very important to pay the loan on time.B: I understand, but I do not have money right now.Can you get some time?Ra: We understand your situation.Can you pay something until next week?B: I will try, but I will not be able to give full EMI.Will half the payment run?Ra: Okay, pay half the payment by next week.What is your plan for the rest?B: I hope I will get a new job by next month.Then I will pay the rest of the arrears.Ra: Okay.So we do this - you submit half EMI by next week, and pay the rest by 15th of next month.Do you accept this?B: Yes, it will be fine.I will try my best to follow this plan.Ra: Very good.I am sending you 

In [21]:
# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MohdS\.cache\huggingface\hub\models--philschmid--bart-large-cnn-samsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [52]:
summary = summarizer(translated_text, max_length=100, min_length=70, do_sample=False)[0]['summary_text']

In [53]:
summary

"Borrower's previous month's EMI has not come yet. B's job is gone and he is looking for a new one. B will pay half of the payment by next week and the rest by the 15th of the next month. Ra will send B an SMS with payment details. B accepts the payment plan. "